In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn .preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import argparse
import cv2
import os
import xgboost as xgb
import imageio

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
def quantify_image(image):
    # compute the HOG feature vector for the input image
    features = feature.hog(image, 
                           orientations=9, 
                           pixels_per_cell=(10, 10), 
                           cells_per_block=(2,2),
                           transform_sqrt=True, 
                           block_norm="L1"
                           )
    return features

In [ ]:
def load_split(path):
    # grab the list of images in the input directory, then initialize the list of data(i.e., images) and class labels
    imagePaths = list(paths.list_images(path))
    data = []
    labels = []    
    # loop over the image paths
    for imagePath in imagePaths:
        # extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]
        # load the input image, convert it to grayscale, and resize it to 200x2000 pixels, ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (200,200))
        # Threshold the image such that the drawing appears as white on a black background
        image = cv2.threshold(image,0,255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)[1]
        # quantify the image
        features = quantify_image(image)
        # update the data and labels lists, respectively
        data.append(features)
        labels.append(label)
    return (np.array(data), np.array(labels))

In [ ]:
dataset_path = r'C:\Users\medical_robot\Documents\Python Scripts\OpenCV+DL\detect-parkinsons\dataset\wave'
trials = 5
trainingPath = os.path.sep.join([dataset_path,"training"])
testingPath = os.path.sep.join([dataset_path, "testing"])

# load the traning and testing data
print("[INFO] loading data...")
(trainX, trainY) = load_split(trainingPath)
(testX, testY) = load_split(testingPath)

# Encode the labels as integers
le = LabelEncoder()
trainY = le.fit_transform(trainY)
testY = le.transform(testY)

#initialize or trials dicitonary
trials = {}

In [ ]:
# loop over the number of trials to run
for i in range(0, trial_num):
    # train the model
    print(f"[INFO] training model {i+1} of {trial_num}...")
    model = RandomForestClassifier(n_estimators=100)
    model.fit(trainX, trainY)
    
    # make predictions on the testing data and initialize a dictionary to store our computed metreics
    predictions = model.predict(testX)
    metrics = {}
    
    # compute the confusion matrix and use it to derive the raw accuracy, ensitivity, and specificity
    cm = confusion_matrix(testY, predictions).flatten()
    (tn, fp, fn, tp) = cm
    metrics["acc"] = (tp + tn) / float(cm.sum())
    metrics["sensitivity"] = tp / float(tp + fn)
    metrics["specificity"] = tn / float(tn + fp)
    
    # loop over the metrics
    for (k,v) in metrics.items():
        # update the trialas dictionary with the list of values for the current metric
        l = trials.get(k, [])
        l.append(v)
        trials[k] = l

# loop over our metrics
for metric in ("acc", "sensitivity", "specificity"):
    # grab the list of values for the current metric, then compute the mean and standard deviation
    values = trials[metric]
    mean = np.mean(values)
    std = np.std(values)
    
    # show the computed metrics for the statistic
    print(metric)
    print('='*len(metric))
    print(f'mean={mean:.4f}, std={std:.4f} \n')  

In [ ]:
# randomly select a few images and then initialize the output image for the montage
testingPaths = list(paths.list_images(testingPath))
idxs = np.arange(0, len(testingPaths))
idxs = np.random.choice(idxs, size=(25,), replace=False)
images = []

# loop over the trsting samples
for i in idxs:
    # load the teting image, clone it, and resize it
    image = cv2.imread(testingPaths[i])
    output = image.copy()
    output = cv2.resize(output, (128,128))
    
    # pre-process the image in the same manner we did earlier
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (200, 200))
    image = cv2.threshold(image, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    
    # quantify the image and make predictions based on the extracted features using the last trained RF
    features = quantify_image(image)
    preds = model.predict([features])
    label = le.inverse_transform(preds)[0]
    
    # draw the colored class label on the output image and add it to the set of output images
    color = (0, 255, 0) if label == "healthy" else (0, 0, 255)
    cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    images.append(output)    

# create a montage using 128x128 "titles" with 5 rows and 5 columns
montage = build_montages(images, (128,128), (5,5))[0]
# show the output montage
cv2.imshow("Output", montage)
cv2.waitKey(0)
cv2.destroyAllWindows()